# A notebook to copy and clean up PIPS netCDF files for upload to the PERiLS EOL repository

In [1]:
%load_ext autoreload
%autoreload 2
import numpy as np
import numpy.ma as ma
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.ticker as ticker
import matplotlib.dates as dates
from mpl_toolkits.axes_grid1 import ImageGrid,make_axes_locatable,host_subplot
#from mpl_toolkits.basemap import Basemap
from datetime import datetime, timedelta
import sys
import os
import pyPIPS.utils as utils
import pyPIPS.thermolib as thermo
import pyPIPS.DSDlib as dsd
#import pyPIPS.disdrometer_module as dis
import pyPIPS.plotmodule as PIPSplot
#import pyPIPS.simulator as sim
import pyPIPS.pips_io as pipsio
import pyPIPS.PIPS as pips
import pyPIPS.parsivel_params as pp
import pyPIPS.parsivel_qc as pqc
import pyPIPS.polarimetric as dualpol
#from pyCRMtools.modules import plotmodule as plotmod
from pyCRMtools.modules import utils as CRMutils
# from pyCRMtools.pycaps import arps_read
# from pyCRMtools.pycaps import pycaps_fields
# from pyCRMtools.pycaps import calvars_radar as radar
import pandas as pd
import xarray as xr
import glob
import numpy.random as random
from scipy.stats import gamma, uniform
from scipy.special import gamma as gammafunc
from scipy import ndimage
from metpy.plots import StationPlot
from metpy.calc import wind_components
from metpy.cbook import get_test_data
from metpy.plots import StationPlot
from metpy.plots.wx_symbols import current_weather, sky_cover
from metpy.units import units
import warnings
warnings.simplefilter('ignore')
%matplotlib notebook


## You are using the Python ARM Radar Toolkit (Py-ART), an open source
## library for working with weather radar data. Py-ART is partly
## supported by the U.S. Department of Energy as part of the Atmospheric
## Radiation Measurement (ARM) Climate Research Facility, an Office of
## Science user facility.
##
## If you use this software to prepare a publication, please cite:
##
##     JJ Helmus and SM Collis, JORS 2016, doi: 10.5334/jors.119



<frozen importlib._bootstrap>:283: DeprecationWarning: the load_module() method is deprecated and slated for removal in Python 3.12; use exec_module() instead


In [2]:
# Read in the original PIPS netcdf files

PIPS_input_base_dir = '/Users/dawson29/Projects/PERiLS/obsdata/2022/PIPS_data_for_EOL/'
PIPS_output_base_dir = '/Users/dawson29/Projects/PERiLS/obsdata/2022/PIPS_data_for_EOL/'
deployment_name = 'IOP3_040522'
PIPS_input_dir = os.path.join(PIPS_input_base_dir, deployment_name, 'netcdf')
PIPS_output_dir = os.path.join(PIPS_output_base_dir, deployment_name, 'netcdf_temp')
if not os.path.exists(PIPS_output_dir):
    os.makedirs(PIPS_output_dir)

# IOP1
# PIPS_names = ['PIPS1A', 'PIPS1B', 'PIPS2A', 'PIPS2B']
# IOP2
# PIPS_names = ['PIPS1A', 'PIPS1B', 'PIPS2A', 'PIPS3B']
# IOP3
PIPS_names = ['PIPS1A', 'PIPS1B', 'PIPS2A', 'PIPS2B', 'PIPS3A', 'PIPS3B']
parsivel_interval = 10
parsivel_filenames = ['parsivel_combined_{}_{}_{:d}s.nc'.format(deployment_name, PIPS_name, parsivel_interval)
                      for PIPS_name in PIPS_names]
parsivel_filepaths = [os.path.join(PIPS_input_dir, parsivel_filename) for parsivel_filename in parsivel_filenames]
output_parsivel_filepaths = [os.path.join(PIPS_output_dir, parsivel_filename) 
                             for parsivel_filename in parsivel_filenames]
conv_filenames = ['conventional_raw_{}_{}.nc'.format(deployment_name, PIPS_name) for PIPS_name in PIPS_names]
conv_filepaths = [os.path.join(PIPS_input_dir, conv_filename) for conv_filename in conv_filenames]
output_conv_filepaths = [os.path.join(PIPS_output_dir, conv_filename) for conv_filename in conv_filenames]
parsivel_ds_dict = {}
conv_ds_dict = {}
for PIPS_name, parsivel_filepath, conv_filepath in zip(PIPS_names, parsivel_filepaths, conv_filepaths):
    parsivel_ds_dict[PIPS_name] = xr.load_dataset(parsivel_filepath)
    conv_ds_dict[PIPS_name] = xr.load_dataset(conv_filepath)

In [3]:
PIPS_to_check = 'PIPS1A'

In [4]:
parsivel_ds = parsivel_ds_dict[PIPS_to_check]
parsivel_ds

<xarray.Dataset>
Dimensions:                    (time: 757, fallspeed_bin: 32, diameter_bin: 32)
Coordinates:
  * time                       (time) datetime64[ns] 2022-04-05T14:45:43 ... ...
    fallspeed                  (fallspeed_bin) float64 0.05 0.15 ... 17.6 20.8
    diameter                   (diameter_bin) float64 0.0625 0.1875 ... 24.5
    min_diameter               (diameter_bin) float64 0.0 0.125 ... 20.0 23.0
    max_diameter               (diameter_bin) float64 0.125 0.25 ... 23.0 26.0
    min_fallspeeds             (fallspeed_bin) float64 0.0 0.1 0.2 ... 16.0 19.2
    max_fallspeeds             (fallspeed_bin) float64 0.1 0.2 0.3 ... 19.2 22.4
    flagged_times              (time) int64 0 0 0 0 0 0 0 0 ... 0 0 0 0 0 0 0 0
Dimensions without coordinates: fallspeed_bin, diameter_bin
Data variables: (12/54)
    precipintensity            (time) float64 3.445 10.51 5.276 ... 5.633 4.436
    precipaccum                (time) float64 0.0 0.03 0.05 ... 36.6 36.62 36.63
    parsivel_dBZ               (time) float64 31.67 37.64 31.89 ... 38.26 39.27
    sample_interval            (time) timedelta64[ns] 00:00:09 ... 00:00:10
    signal_amplitude           (time) float64 2.017e+04 2.016e+04 ... 1.322e+04
    pcount                     (time) int64 56 70 158 120 124 ... 21 33 29 31 21
    ...                         ...
    rainrate_derived_roqc      (time) float64 3.375 5.433 5.853 ... 5.863 4.587
    pcount_derived_roqc        (time) float64 55.0 70.0 121.0 ... 27.0 31.0 21.0
    reflectivity_derived_roqc  (time) float64 31.21 34.63 31.88 ... 38.26 39.27
    D0_roqc                    (time) float64 0.001363 0.001463 ... 0.002439
    Dm43_roqc                  (time) float64 0.00138 0.00153 ... 0.002495
    sigma_roqc                 (time) float64 0.000448 0.0005446 ... 0.001017
Attributes:
    nominal sample interval:  10 seconds
    probe_name:               PIPS1A
    parsivel_angle:           -45.0
    deployment_name:          IOP3_040522
    location:                 (32.26733333333334, -86.30216666666664, 76.6704...
    starting_time:            20220405144543
    ending_time:              20220405165143
    DSD_interval:             10.0

In [10]:
conv_ds = conv_ds_dict[PIPS_to_check]
conv_ds

<xarray.Dataset>
Dimensions:      (time: 7579)
Coordinates:
  * time         (time) datetime64[ns] 2022-04-05T14:45:30 ... 2022-04-05T16:...
Data variables: (12/24)
    voltage      (time) float64 12.62 12.58 12.58 12.58 ... 12.39 12.4 12.4
    winddirrel   (time) float64 113.0 127.0 126.0 150.0 ... 232.0 241.0 231.0
    windspd      (time) float64 0.29 0.54 0.69 0.51 0.55 ... 0.57 0.58 0.49 0.38
    winddiag     (time) float64 0.0 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
    fasttemp     (time) float64 20.4 20.38 20.35 20.33 ... 18.21 18.22 18.22
    slowtemp     (time) float64 21.83 21.76 21.85 21.76 ... 18.19 18.12 18.16
    ...           ...
    winddirabs   (time) float64 113.0 175.9 174.9 198.9 ... 103.0 112.0 102.0
    dewpoint     (time) float64 18.25 18.2 18.34 18.26 ... 17.85 17.77 17.81
    RH_derived   (time) float64 87.45 87.29 88.27 87.89 ... 97.78 97.19 97.43
    pt           (time) float64 nan 293.8 293.7 293.7 ... 291.5 291.6 291.6
    qv           (time) float64 nan 0.01336 0.01349 ... 0.01306 0.01299 0.01302
    rho          (time) float64 nan 1.174 1.174 1.174 ... 1.184 1.184 1.184
Attributes:
    probe_name:       PIPS1A
    parsivel_angle:   -45.0
    deployment_name:  IOP3_040522
    location:         (32.26733333333334, -86.30216666666664, 76.67049158606068)
    starting_time:    20220405144543
    ending_time:      20220405165143

In [ ]:
# Make a dictionary of the variable names and their units for those that don't already have them
parsivel_var_units_dict = {
    'windspdavgvec': 'meters per second',
    'winddirunitavgvec': 'degrees',
    'uavg': 'meters per second',
    'vavg': 'meters per second',
    'rho': 'kilograms per cubic meter',
    'fallspeed': 'meters per second',
    'diameter': 'millimeters',
    'min_diameter': 'millimeters',
    'max_diameter': 'millimeters',
    'min_fallspeeds': 'meters per second',
    'max_fallspeeds': 'meters per second',
    'VD_matrix': 'count',
    'ND': 'number per cubic meter per millimeter',
    'VD_matrix_qc': 'count',
    'ND_qc': 'number per cubic meter per millimeter',
    'VD_matrix_roqc': 'count',
    'ND_roqc': 'number per cubic meter per millimeter',
    'pcount_derived': 'count',
    'pcount_derived_qc': 'count',
    'pcount_derived_roqc': 'count'
}

In [ ]:
# Now go through and update the units attribute for the needed variables using this dictionary

for PIPS_name in PIPS_names:
    for var_name in parsivel_ds_dict[PIPS_name].data_vars:
        if var_name in parsivel_var_units_dict:
            unit = parsivel_var_units_dict[var_name]
            parsivel_ds_dict[PIPS_name][var_name].attrs['units'] = unit

In [ ]:
# Now save to new output directory
for PIPS_name, output_parsivel_filepath in zip(PIPS_names, output_parsivel_filepaths):
    print("Saving {}".format(output_parsivel_filepath))
    parsivel_ds_dict[PIPS_name].to_netcdf(output_parsivel_filepath)

In [ ]:
# Now do the same for the conventional dataset

conv_var_units_dict = {
    'pt': 'degrees Kelvin',
    'qv': 'kg per kg (unitless)',
    'rho': 'kg per cubic meter'
}

In [ ]:
# Now go through and update the units attribute for the needed variables using this dictionary

for PIPS_name in PIPS_names:
    for var_name in conv_ds_dict[PIPS_name].data_vars:
        if var_name in conv_var_units_dict:
            unit = conv_var_units_dict[var_name]
            conv_ds_dict[PIPS_name][var_name].attrs['units'] = unit
            

In [ ]:
# Now save to new output directory
for PIPS_name, output_conv_filepath in zip(PIPS_names, output_conv_filepaths):
    print("Saving {}".format(output_conv_filepath))
    conv_ds_dict[PIPS_name].to_netcdf(output_conv_filepath)